## Kubernetes Clusterwide SSL Demo

### Setup

Create a Kind cluster by using an ansible playbook from the project `ansible` folder.


In [ ]:
!cd ../ansible && ansible-playbook playbooks/kind-cluster.yaml

In [ ]:
!cd ../ansible && ansible-playbook playbooks/setup-ecosystem.yaml

In [48]:
!helm upgrade --install seldon-core-v2-crds  ../k8s/helm-charts/seldon-core-v2-crds -n seldon-mesh

Release "seldon-core-v2-crds" does not exist. Installing it now.
NAME: seldon-core-v2-crds
LAST DEPLOYED: Wed Jun 21 11:03:14 2023
NAMESPACE: seldon-mesh
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [49]:
!kubectl create namespace ns1

Error from server (AlreadyExists): object is being deleted: namespaces "ns1" already exists


In [5]:
!kubectl create namespace ns2

Error from server (AlreadyExists): namespaces "ns2" already exists


In [6]:
!helm install seldon-v2-certs ../k8s/helm-charts/seldon-core-v2-certs/ -n ns1

NAME: seldon-v2-certs
LAST DEPLOYED: Wed Jun 21 10:29:32 2023
NAMESPACE: ns1
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [7]:
!helm install seldon-v2-certs ../k8s/helm-charts/seldon-core-v2-certs/ -n ns2

NAME: seldon-v2-certs
LAST DEPLOYED: Wed Jun 21 10:29:34 2023
NAMESPACE: ns2
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [8]:
!cat ../k8s/samples/values-tls-dataplane-controlplane-example.yaml

security:
  controlplane:
    protocol: SSL
    ssl:
      server:
        secret: seldon-controlplane-server
        clientValidationSecret: seldon-controlplane-client
      client:
        secret: seldon-controlplane-client
        serverValidationSecret: seldon-controlplane-server
  envoy:
    protocol: SSL
    ssl:
      upstream:
        server:
          secret: seldon-upstream-server
          clientValidationSecret: seldon-upstream-client
        client:
          secret: seldon-upstream-client
          serverValidationSecret: seldon-upstream-server
      downstream:
        client:
          serverValidationSecret: seldon-downstream-server
        server: 
          secret: seldon-downstream-server


In [9]:
!helm install seldon-v2 ../k8s/helm-charts/seldon-core-v2-setup/ -n seldon-mesh --set controller.clusterwide=true --values ../k8s/samples/values-tls-dataplane-controlplane-example.yaml

NAME: seldon-v2
LAST DEPLOYED: Wed Jun 21 10:29:40 2023
NAMESPACE: seldon-mesh
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [10]:
!helm install seldon-v2-runtime ../k8s/helm-charts/seldon-core-v2-runtime  -n ns1

NAME: seldon-v2-runtime
LAST DEPLOYED: Wed Jun 21 10:30:26 2023
NAMESPACE: ns1
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [11]:
!helm install seldon-v2-servers ../k8s/helm-charts/seldon-core-v2-servers  -n ns1

NAME: seldon-v2-servers
LAST DEPLOYED: Wed Jun 21 10:30:41 2023
NAMESPACE: ns1
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [12]:
!helm install seldon-v2-runtime ../k8s/helm-charts/seldon-core-v2-runtime  -n ns2

NAME: seldon-v2-runtime
LAST DEPLOYED: Wed Jun 21 10:30:44 2023
NAMESPACE: ns2
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [13]:
!helm install seldon-v2-servers ../k8s/helm-charts/seldon-core-v2-servers  -n ns2

NAME: seldon-v2-servers
LAST DEPLOYED: Wed Jun 21 10:31:00 2023
NAMESPACE: ns2
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [14]:
!kubectl wait --for condition=ready --timeout=300s server --all -n ns1

server.mlops.seldon.io/mlserver condition met
server.mlops.seldon.io/triton condition met


In [15]:
!kubectl wait --for condition=ready --timeout=300s server --all -n ns2

server.mlops.seldon.io/mlserver condition met
server.mlops.seldon.io/triton condition met


### Setup TLS Config

In [16]:
MESH_IP=!kubectl get svc seldon-mesh -n ns1 -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
MESH_IP_NS1=MESH_IP[0]
import os
os.environ['MESH_IP_NS1'] = MESH_IP_NS1
MESH_IP_NS1

'172.21.255.2'

In [17]:
MESH_IP=!kubectl get svc seldon-mesh -n ns2 -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
MESH_IP_NS2=MESH_IP[0]
import os
os.environ['MESH_IP_NS2'] = MESH_IP_NS2
MESH_IP_NS2

'172.21.255.4'

In [18]:
!seldon config add tls ${PWD}/config-dataplane-tls.json

In [19]:
!seldon config activate tls

### Launch model in namespace ns1

In [35]:
!kubectl create -f ./models/sklearn-iris-gs.yaml -n ns1

model.mlops.seldon.io/iris created


In [36]:
!kubectl wait --for condition=ready --timeout=300s model --all -n ns1

model.mlops.seldon.io/iris condition met


In [37]:
!seldon model infer iris --inference-host ${MESH_IP_NS1}:80 \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

{
	"model_name": "iris_1",
	"model_version": "1",
	"id": "d7ba2bcb-b901-4c39-a91e-0e19b67a9d19",
	"parameters": {},
	"outputs": [
		{
			"name": "predict",
			"shape": [
				1,
				1
			],
			"datatype": "INT64",
			"parameters": {
				"content_type": "np"
			},
			"data": [
				2
			]
		}
	]
}


### Launch model in namespace ns2

In [38]:
!kubectl create -f ./models/sklearn-iris-gs.yaml -n ns2

model.mlops.seldon.io/iris created


In [39]:
!kubectl wait --for condition=ready --timeout=300s model --all -n ns2

model.mlops.seldon.io/iris condition met


In [40]:
!seldon model infer iris --inference-host ${MESH_IP_NS2}:80 \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

{
	"model_name": "iris_1",
	"model_version": "1",
	"id": "cd07f18a-a408-4365-8924-75e4db3bfaf9",
	"parameters": {},
	"outputs": [
		{
			"name": "predict",
			"shape": [
				1,
				1
			],
			"datatype": "INT64",
			"parameters": {
				"content_type": "np"
			},
			"data": [
				2
			]
		}
	]
}


## TearDown

In [41]:
!kubectl delete -f ./models/sklearn-iris-gs.yaml -n ns1

model.mlops.seldon.io "iris" deleted


In [42]:
!kubectl delete -f ./models/sklearn-iris-gs.yaml -n ns2

model.mlops.seldon.io "iris" deleted


In [43]:
!helm delete seldon-v2-servers -n ns1
!helm delete seldon-v2-runtime -n ns1
!helm delete seldon-v2-certs -n ns1

release "seldon-v2-servers" uninstalled
release "seldon-v2-runtime" uninstalled
release "seldon-v2-certs" uninstalled


In [44]:
!helm delete seldon-v2-runtime -n ns2
!helm delete seldon-v2-runtime -n ns2
!helm delete seldon-v2-certs -n ns2

release "seldon-v2-runtime" uninstalled
Error: uninstall: Release not loaded: seldon-v2-runtime: release: not found
release "seldon-v2-certs" uninstalled


In [45]:
!helm delete seldon-v2  -n seldon-mesh

release "seldon-v2" uninstalled


In [46]:
!helm delete seldon-core-v2-crds -n seldon-mesh

release "seldon-core-v2-crds" uninstalled


In [ ]:
!kubectl delete namespace ns1
!kubectl delete namespace ns2

namespace "ns1" deleted
